In [1]:
import re
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


# HW3: Computing Weight Counts in Keras CNN Models

In [2]:
model_desc = """model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))"""

In [3]:
def process_desc(model_desc):
    filtered = []
    for line in model_desc.split('\n'):
        if affects_param_count(line):
            filtered.append(process_line(line))
    return filtered

In [4]:
def affects_param_count(line):
     return 'Dropout' not in line and 'Sequential' not in line

In [5]:
def process_line(line):
    line = line[line.find('(') + 1: line.rfind(')')]
    return line.replace(' ', '')

In [6]:
def process_layer(layer, weights, input_shape):
    params = layer[layer.find('(') + 1: layer.rfind(')')]        
    
    if 'Conv2D' in layer:
        # find the count of kernels (first arg)
        kernel_count = int(params[:params.find(',')])
        # find the shape of kernel
        ksize_search = re.search('kernel_size=\((\d+),(\d+)\)', params)
        kernel_shape = (int(ksize_search.group(1)), int(ksize_search.group(2)))
        # update weight count including bias weights
        weights += input_shape[2] * kernel_shape[0] * kernel_shape[1] * kernel_count + kernel_count
        input_shape = (input_shape[0] - kernel_shape[0] + 1, # reshape because of 'valid' padding
                       input_shape[1] - kernel_shape[1] + 1, # reshape because of 'valid' padding
                       kernel_count)
    
    elif 'MaxPooling2D' in layer:
        input_shape = (input_shape[0] // 2, input_shape[1] // 2, input_shape[2])
    
    elif 'Flatten' in layer:
        input_shape = (input_shape[0] * input_shape[1] * input_shape[2], )
    
    elif 'Dense' in layer:
        layer_size = int(params[:params.find(',')])
        # update weight count including bias weights
        weights += input_shape[0] * layer_size + layer_size
        input_shape = (layer_size,)

    return weights, input_shape

In [7]:
print('layer', '\t\t+ weights', '\toutput shape')
print('--------------------------------------------')

weights = 0
input_shape = (128, 128, 3)
layers = process_desc(model_desc)
for layer in layers:
    old_w = weights
    weights, input_shape = process_layer(layer, weights, input_shape)
    print(layer[:5], '\t\t+', weights - old_w, '\t\t', input_shape)

print('--------------------------------------------')
print('Total trainable weights:', weights)

layer 		+ weights 	output shape
--------------------------------------------
Conv2 		+ 896 		 (126, 126, 32)
Conv2 		+ 18496 		 (124, 124, 64)
MaxPo 		+ 0 		 (62, 62, 64)
Flatt 		+ 0 		 (246016,)
Dense 		+ 31490176 		 (128,)
Dense 		+ 1290 		 (10,)
--------------------------------------------
Total trainable weights: 31510858


### Comparing with the output of summary()

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 126, 126, 32)      896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 246016)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               31490176  
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

### Param counts match!